# Lab 1: Langfuse トレーシング

このラボでは、Langfuse トレーシングを使用して LLM アプリケーションの実行をログに記録し分析する方法を学びます。Langfuse は AWS でのセルフホストをサポートしており、[クラウドバージョン](https://cloud.langfuse.com/)も利用可能です。Langfuse の [トレーシング](https://langfuse.com/docs/tracing)は、LLM アプリケーションの実行をログに記録し分析する方法であり、以下のリファレンスでは使用されるデータモデルの詳細な概要が提供されています。これは OpenTelemetry にインスパイアされています。

## [トレースとオブザベーション](https://langfuse.com/docs/tracing-data-model)

トレースは通常、単一のリクエストまたは操作を表します。それには、関数の全体的な入出力、およびユーザー、セッション、タグなどのリクエストに関するメタデータが含まれます。通常、トレースはアプリケーションの単一の API 呼び出しに対応します。

各トレースには、実行の個々のステップをログに記録するための複数のオブザベーションが含まれます。

- オブザベーションには異なるタイプがあります：
    - イベントは基本的な構成要素です。トレース内の個別のイベントを追跡するために使用されます。
    - スパンは、トレース内の作業単位の期間を表します。
    - ジェネレーションは、AI モデルの生成をログに記録するために使用されるスパンのことです。モデル、プロンプト、および完了に関する追加属性が含まれます。ジェネレーションの場合、[トークンの使用とコスト](https://langfuse.com/docs/model-usage-and-cost)が自動的に計算されます。
- オブザベーションはネスト可能です。

![Trace and Observations](./images/trace-observation.png)
![Trace and Observations UI](./images/trace-observation-ui.png)

## [セッション](https://langfuse.com/docs/tracing-data-model)
オプションで、トレースをセッションにグループ化できます。セッションは、同じユーザーインタラクションの一部であるトレースをグループ化するために使用されます。一般的な例として、チャットインターフェースのスレッドが挙げられます。トレースにセッションを追加するには、[セッションドキュメント](https://langfuse.com/docs/sessions)を参照してください。

![Trace and Sessions](./images/trace-sessions.png)
![Trace and Sessions UI](./images/trace-sessions-ui.png)


## [スコア](https://langfuse.com/docs/tracing-data-model)

トレースとオブザベーションは、[スコア](https://langfuse.com/docs/scores/overview)を使用して評価できます。スコアは評価メトリクスを保存する柔軟なオブジェクトであり、次のようになります：

- 数値、カテゴリー、またはブール値
- トレースに関連付けられる (必須)
- 特定のオブザベーションにリンクされる (オプション)
- 追加のコンテキストのためのコメントで注釈付けされる
- スコア設定スキーマに対して検証される (オプション)

![Trace and Scores](./images/trace-scores.png)

開始するには、[スコアドキュメント](https://langfuse.com/docs/scores/overview)を参照してください。スコアタイプと属性の詳細については、[スコアデータモデルドキュメント](https://langfuse.com/docs/scores/data-model)を参照してください。



## 前提条件

> カーネルを選択していない場合は、右上隅にある「Select Kernel」ボタンをクリックし、Python Environmentsを選択して「.venv (Python 3.9.20) .venv/bin/python Recommended」を選択してください。

> 各ノートブックセルを実行するには、Shift + Enterを押してください。

> ℹ️ AWS が提供する一時アカウントを使用してインストラクター主導のワークショップに参加している場合は、これらの前提条件ステップを**スキップ**できます

### 依存関係と環境変数

langfuse と boto3 を使用します：
- Langfuse Python SDK とセルフホスティングデプロイメントを使用して、モデル呼び出しのトレーシング、プロンプト/モデル設定の管理、評価の実行により LLM アプリケーションをデバッグおよび改善します。
- boto3 SDK を使用して、Amazon Bedrock または Amazon SageMaker 上のモデルと対話します。

必要な Python SDK をインストールするために以下のコマンドを実行してください。
> AWS が主催するイベントで提供された AWS アカウントを使用している場合は、インストールをスキップしてください


In [ ]:
# AWS ワークショップ環境を使用していない場合は、以下の行のコメントを外して依存関係をインストールしてください
# %pip install -q langfuse boto3  --upgrade

.env ファイルで Langfuse プロジェクトと API キーをセットアップし、セルフホストまたはクラウドの Langfuse 環境に接続するための前提条件が完了していることを確認してください。


In [ ]:
# すでに VS Code サーバーの .env で環境変数を定義している場合は、以下のセルはスキップしてください。
# langfuse 用の環境変数を定義してください。
# これらの値は Langfuse で API キーを作成する際に確認することができます。
# import os
# os.environ["LANGFUSE_SECRET_KEY"] = "xxxx" # Langfuse プロジェクトのシークレットキー
# os.environ["LANGFUSE_PUBLIC_KEY"] = "xxxx" # Langfuse プロジェクトのパブリックキー
# os.environ["LANGFUSE_HOST"] = "xxx" # Langfuse ドメイン

詳細については [Langfuse ドキュメント](https://langfuse.com/docs/get-started) を確認してください。

## 初期化と認証チェック
以下のセルを実行して、共通ライブラリとクライアントを初期化してください。


In [ ]:
# import all the necessary packages
import sys
import os
from typing import Optional

import boto3
from langfuse import Langfuse
from langfuse.decorators import langfuse_context, observe
from langfuse.model import PromptClient

Amazon Bedrock クライアントと Amazon Bedrock ランタイム クライアントを作成します。このラボではリージョンが us-west-2 であることを確認してください。期待される結果は次の出力が表示されることです：

```
Found Nova model: US Nova Pro - us.amazon.nova-pro-v1:0
Found Nova model: US Nova Lite - us.amazon.nova-lite-v1:0
Found Nova model: US Nova Micro - us.amazon.nova-micro-v1:0
```

> us-west-2 の Nova モデルはクロスリージョン推論 (CRIS) でのみ呼び出すことができるため、model_id には "us." プレフィックスが付いています。これは CRIS 呼び出しであることを示しています。これにより、モデル呼び出しに遅延が加わる可能性があります。

In [ ]:

# Amazon Bedrock のコンフィグへのアクセスに利用
# このラボではリージョンを us-west-2 に設定
bedrock = boto3.client(service_name="bedrock", region_name="us-west-2")

# Amazon Nova モデルがこのリージョンで使えるかチェック
models = bedrock.list_inference_profiles()
nova_found = False
for model in models["inferenceProfileSummaries"]:
    if (
        "Nova Pro" in model["inferenceProfileName"]
        or "Nova Lite" in model["inferenceProfileName"]
        or "Nova Micro" in model["inferenceProfileName"]
    ):
        print(
            f"Found Nova model: {model['inferenceProfileName']} - {model['inferenceProfileId']}"
        )
        nova_found = True
if not nova_found:
    raise ValueError(
        "No Nova models found in available models. Please ensure you have access to Nova models."
    )

Langfuse クライアントを初期化し、認証情報が有効であることを確認します。

In [ ]:
from langfuse import Langfuse

# langfuse クライアント
langfuse = Langfuse()
if langfuse.auth_check():
    print("Langfuse has been set up correctly")
    print(f"You can access your Langfuse instance at: {os.environ['LANGFUSE_HOST']}")
else:
    print(
        "Credentials not found or invalid. Check your Langfuse API key and host in the .env file."
    )

### Amazon Bedrock Converse API の Langfuse ラッパー
Amazon Bedrock Converse API を使用して、Amazon Bedrock モデルとの間でメッセージを送受信する対話型アプリケーションを作成できます。例えば、複数回のやり取りにわたって会話を維持し、役立つテクニカルサポートアシスタントなど、独自のニーズに合わせてパーソナリティやトーンをカスタマイズしたチャットボットを作成できます。

Converse API を使用するには、Converse または ConverseStream（ストリーミング応答用）操作を使用してモデルにメッセージを送信します。既存の基本推論操作（InvokeModel または InvokeModelWithResponseStream）を会話アプリケーションに使用することも可能です。ただし、Converse API の使用を推奨します。これは、メッセージをサポートするすべての Amazon Bedrock モデルで動作する一貫した API を提供するためです。つまり、一度コードを書いて異なるモデルで使用できます。モデルに固有の推論パラメータがある場合、Converse API ではモデル固有の構造でそれらの固有パラメータを渡すこともできます。

詳細については、[Converse API 操作で会話を実行する](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)を参照してください。

In [ ]:
sys.path.append(os.path.abspath('..'))  # 親ディレクトリをパスに追加
from config import GUARDRAIL_CONFIG, MODEL_CONFIG
from utils import converse, converse_tool_use

### チャットの例

#### Converse API ラッパーを呼び出すためのヘルパー関数を定義する

> ワークショップスタジオの紹介セクションにある [Langfuse セットアップ](https://catalog.workshops.aws/genaiops-langfuse/ja-JP/00-introduction/langfuse-setup) で言及されている Nova カスタムモデルの価格設定を設定したことを確認してください。

In [ ]:
@observe(name="Simple Chat")
def simple_chat(
    model_config: dict,
    messages: list,
    prompt: PromptClient = None,
    use_guardrails: bool = False,
) -> dict:
    """
    指定されたモデル設定を使ってシンプルなチャットインタラクションを実行する。

    Args:
        model_config (dict): チャットモデルの設定パラメータ
        messages (list): 処理されるメッセージ辞書のリスト
        prompt (PromptClient, optional): オプションのプロンプトクライアントで高度な処理が可能
        use_guardrails (bool, optional): True の場合、追加のガードレール構成を適用

    Returns:
        dict: converse' 関数呼び出しからの応答
    """
    config = model_config.copy()
    if use_guardrails:
        config["guardrailConfig"] = GUARDRAIL_CONFIG
    return converse(messages=messages, prompt=prompt, **config)

#### Use Case 1
let's start with a single turn chat use case and use Nova Pro as the default model.

In [ ]:
# Decorator to observe and track this function execution in Langfuse
@observe(name="Single Turn Example")
def chat_single_model(
    messages: list, model_type: str = "nova_pro", use_guardrails: bool = False
) -> dict:
    """
    指定されたひとつの Amazon Nova モデルを使って、1 ターンのチャットインタラクションを実行

    Args:
        messages (list): ユーザーの入力クエリ
        model_type (str): 利用する Nova モデル (nova_pro, nova_lite, or nova_micro)
        use_guardrails (bool): モデル呼び出しにガードレールを適用するか

    Returns:
        dict: モデルからのレスポンスとステータスコード
    """
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        tags=["lab1", "single-turn"],
    )

    response = simple_chat(
        model_config=MODEL_CONFIG[model_type],
        messages=messages,
        use_guardrails=use_guardrails,
    )

    return {"model": model_type, "response": response, "statusCode": 200}


# チャット API をテストするためにサンプルリクエストを作成する
# 高級リゾートのチェックインプロセスについて質問する
print(
    chat_single_model(
        [
            {
                "role": "user",
                "content": "高級リゾートでゲストをチェックインするプロセスを、ステップバイステップで説明して。",
            }
        ]
    )
)

# 自動の flush を待つのではなく、
# Langfuse へのトレースデータの送信を強制する
langfuse_context.flush()

In [ ]:
print(
    f"Langfuse のダッシュボードで、トレース、モデルのコスト、モデルの使用状況の概要を確認できます:\n{os.environ['LANGFUSE_HOST']}"
)

![Langfuse Dashboard](./images/langfuse-dashboard-use-case-1.png)


詳細なトレースは **Traces** セクションにあり、トレースをクリックすると詳細なトレースを表示できます。このトレースにはいくつかの重要な洞察があります。

- 入力トークンは 12 で、出力トークンは 662 で、合計 675 トークンが使用されています

- 使用されたモデルは us.amazon.nova-pro-v1:0 です

- 温度、最大トークン数などのモデルパラメータが表示されます

- 最も重要なのは、この呼び出しの総コストは $0.002129 です

![Langfuse Dashboard](./images/langfuse-trace-use-case-1.png)

#### ユースケース 2
このユースケースでは、1 つのセッション内で単一のトレースを実行し、比較のために異なる Nova モデルのバリアントを使用して 3 つの異なるオブザベーションを実行します。

In [ ]:
@observe(name="Multi-Turn Example")
def chat_compare_models(
    messages: list,
    model_types: list = ["nova_pro", "nova_lite", "nova_micro"],
    use_guardrails: bool = False,
) -> dict:
    """
    比較のために、すべての Amazon Nova モデルで同じクエリを実行

    Args:
        messages (list): ユーザーの入力クエリ
        model_types (list): 利用する Nova モデル (nova_pro, nova_lite, or nova_micro)
        use_guardrails (bool): モデル呼び出しにガードレールを適用するか
    Returns:
        dict: すべてのモデルからのレスポンスとステータスコード
    """
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="model-comparison",
        tags=["lab1", "model-comparison"],
    )

    responses = {}
    for model_type in model_types:
        responses[model_type] = simple_chat(
            model_config=MODEL_CONFIG[model_type],
            messages=messages,
            use_guardrails=use_guardrails,
        )

    return {"responses": responses, "statusCode": 200}


# ユーザーリクエスト
print(
    chat_compare_models(
        [
            {
                "role": "user",
                "content": "高級リゾートでゲストをチェックインするプロセスを、ステップバイステップで説明して。",
            }
        ]
    )
)

langfuse_context.flush()

そのため、複数のオブザベーションを 1 つのトレースにまとめ、各オブザベーションのコストと使用量を確認することができます。Nova Micro はコストが最も低く、応答時間が最も速いことがわかります。

![langfuse-traces-use-case-2](./images/langfuse-trace-use-case-2.png)

#### ユースケース 3
今回は、`retrieve_context` と呼ばれるダミーの検索関数を使用して RAG ユースケースをシミュレートします。これは静的なコンテキストを返すダミー関数です。
検索関数からのコンテキストをシステムプロンプトの一部として渡すことで、`simple_chat` 関数を再利用してモデルとチャットします。

In [ ]:
CONTEXT = """2025年1月1日
シドニー: 24 度
ニューヨーク: 13 度
東京: 11 度"""


@observe(name="Dummy Retrieval")
def retrieve_context(query: str) -> str:
    """クエリに対して静的なコンテキストを取得"""
    return CONTEXT


@observe(name="RAG Example")
def rag_api(query: str) -> dict:
    """
    静的なコンテキストを利用して検索拡張生成 (RAG) クエリを実行

    Args:
        query (str): ユーザークエリ

    Returns:
        dict: モデルからのレスポンスとステータスコード
    """
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="rag-session",
        tags=["lab1", "rag-example"],
    )

    context = retrieve_context(query)
    messages = [
        {
            "content": f"Context: {context}\n上記のコンテキストに基づいて以下の質問に回答してください:",
            "role": "system",
        },
        {"content": query, "role": "user"},
    ]
    response = simple_chat(model_config=MODEL_CONFIG["nova_pro"], messages=messages)

    return {"response": response, "statusCode": 200}


# ユーザーリクエスト
print(rag_api("シドニーの天気はいかがですか？何かコメントはありますか？"))

langfuse_context.flush()

トレースを見ると、検索関数が呼び出され、コンテキストがシステムプロンプトの一部としてモデルに渡されるのがわかります。細かいモデルの呼び出しログを見ると、システムプロンプトとユーザープロンプトの両方を受け取り、レスポンスを返していることがわかります。

![langfuse-traces-use-case-3](./images/langfuse-trace-use-case-3.png)


#### ユースケース 4

画像サポートによるマルチモーダル機能

現代の AI システムは、テキスト、画像、音声など、さまざまな種類のデータ入力を処理し理解できるマルチモーダル機能を採用することが増えています。この例では、Langfuse が画像ベースの入力のトレーシングをサポートする方法を示します。これは特に以下に役立ちます：

1. **画像分析**：視覚的なコンテンツの解釈と説明
2. **視覚的質問応答**：画像のコンテキストに基づく質問への回答
3. **ドキュメント処理**：スキャンされたドキュメントや画像から情報の抽出
4. **コンテンツモデレーション**：不適切またはセンシティブな視覚的コンテンツの識別

実装では、画像の URL がユーザープロンプトの一部として渡される構造化メッセージ形式を使用し、モデルがテキストクエリと視覚情報の両方を同時に処理できるようにします。この機能は、以下のようなアプリケーションで特に有用です：

- E コマース製品認識
- 医療画像分析
- ソーシャルメディアコンテンツの理解

Langfuse のトレーシング機能はこれらのマルチモーダルインタラクションにまで及び、モデルが画像入力をどのように処理し応答するかの可視性を提供します。これは、これらの複雑なシステムのデバッグと改善に不可欠です。

このユースケースでは、画像をユーザープロンプトの一部として渡し、モデルがテキストと画像の両方を処理し、langfuse がプロセス全体をトレースします。

In [ ]:
@observe(name="Multi-Modal Image Example")
def vision_api(
    query: str,
    image_url: str,
) -> Optional[str]:
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="vision-session",
        tags=["lab1", "vision-example"],
    )

    messages = [
        {
            "role": "system",
            "content": "あなたは、画像を解釈して説明するように訓練された AI です。",
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        },
    ]

    return {
        "response": simple_chat(
            model_config=MODEL_CONFIG["nova_pro"], messages=messages
        ),
        "statusCode": 200,
    }


# image source: https://www.aboutamazon.com/news/aws/aws-reinvent-2024-keynote-live-news-updates
print(
    vision_api(
        query="この画像では何が起こっていますか？",
        image_url="https://amazon-blogs-brightspot.s3.amazonaws.com/df/82/368cb270402e9739f04905ea9b19/swami-bedrock.jpeg",
    )
)

langfuse_context.flush()

Langfuse は画像入力に対するトレースもサポートしており、モデルが画像を入力とするマルチモーダルなユースケースに非常に便利です。

![langfuse-traces-use-case-4](./images/langfuse-trace-use-case-4.png)


## Langfuse トレーシングを使用した Tool Use
Tool Use により、AI モデルは外部関数や API と対話し、純粋なテキスト生成を超えて機能を拡張できます。これは特に以下に役立ちます：
- リアルタイムデータへのアクセス（例：天気、株価）
- 複雑な計算の実行
- 外部システムとの統合
- 非構造化ソース（例：テキスト、画像）から構造化データの抽出

### ユースケース 1

以下の例は、天気情報ツールの実装を示しています。ユーザーが天気について質問すると、モデルは：

1. 天気データの必要性を認識します
2. 構造化されたツール定義を通じて場所/単位パラメータを抽出します
3. `get_current_weather` ツールを使用してフォーマットされた応答を返します

期待される結果：モデルはサンフランシスコを場所として、摂氏を好みの単位として識別し、構造化されたツール応答を返しながら、Langfuse で完全なトレースの可視性を維持します。

In [ ]:
@observe(name="Tool Use Example")
def tool_use_api(query: str) -> list:
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="tool-use-session",
        tags=["lab1", "tool-use"],
    )

    messages = [{"role": "user", "content": query}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "与えられたロケーションの現在の天気を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "都市と州、例: San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    return {
        "response": converse_tool_use(
            messages, tools, tool_choice="auto", **MODEL_CONFIG["nova_pro"]
        ),
        "statusCode": 200,
    }


print(tool_use_api(query="サンフランシスコの天気はどうですか？摂氏で答えてください。"))

langfuse_context.flush()

![langfuse-traces-tool-use](./images/langfuse-trace-tool-use.png)

### ユースケース 2
以下の例は、マルチモーダルドキュメント書き起こしツールの実装を示しています。ユーザーがドキュメントの転写を要求すると、モデルは：

1. 請求書書き起こしのスキーマを認識します
2. 構造化されたツール定義を通じて画像から構造化データを抽出します
3. `dependentSchemas` を適用して、抽出に対する追加の分類と推論を提供します

期待される結果：モデルは請求書からすべてのメタデータと明細項目を抽出し、各フィールドの分類と推論を含む JSON 形式の構造化データを出力する必要があります。

In [ ]:
system_prompt = """
<instructions>
  - JSON レスポンス内のクォートを必ずエスケープしてください
  - フィールド値が欠損していたら "" を返してください
  - すべての <document/> フィールドに dependentSchemas を適用してください
</instructions>

<document>
{
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "/schemas/document",
    "type": "object",
    "description": "A document with the fields to transcribe",
    "properties": {
        "doc_type": { "properties":{"value":{"type":"string"}}, "description": "Type of Document: Receipt" },
        "receipt_number": { "properties":{"value":{"type":"string"}}, "description": "The receipt number or other identifier number" },
        "doc_amount_total": { "properties":{"value":{"type":"number"}}, "description": "The total receipt amount" },
        "currency": { "properties":{"value":{"type":"string"}}, "description": "AUD/USD/CAD" },
        "vendor_business_number": { "properties":{"value":{"type":"string"}}, "description": "Vendor's business identification number e.g. ABN" },
        "vendor_name": { "properties":{"value":{"type":"string"}}, "description": "Business name issueing the receipt" },
        "vendor_address": { "properties":{"value":{"type":"string"}}, "description": "Vendor's site address" },
        "vendor_phone": { "properties":{"value":{"type":"string"}}, "description": "Vendor's phone number" },
        "payment_method": { "properties":{"value":{"type":"string"}}, "description": "The payment type, e.g. EFTPOS, Card" },
        "date_issued": { "properties":{"value":{"format": "YYYY-MM-DDThh:mm:ss"}}, "description": "Date document was issued"},
        "line_items_amount_total": { "properties":{"value":{"type":"number"}}, "description": "Calculated sum of line item's line_amount fields" }
    },
    "dependentSchemas": {
        "value": {
            "properties": {
                "inference": { "type": "integer", "description": "0=EXPLICIT|1=DERIVED|2=MISSING|3=OTHER" },
                "source": { "type": "string", "description": "Source locations in the document for explicit and derived fields" }
            }
        }
    }
}
<document/>
"""


@observe(name="Vision Tool Use Example")
def vision_tool_use_api(
    query: str,
    image_url: str,
) -> list:
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="tool-use-session",
        tags=["lab1", "tool-use"],
    )

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        },
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "transcribe_documents",
                "description": "Extract all <document/> fields with the highest accuracy following <instructions/>",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "documents": {
                            "type": "array",
                            "items": {"$ref": "/schemas/document"},
                        },
                    },
                    "required": ["documents"],
                },
            },
        }
    ]

    return {
        "response": converse_tool_use(
            messages, tools, tool_choice="auto", **MODEL_CONFIG["nova_pro"]
        ),
        "statusCode": 200,
    }


# image source: https://aws.amazon.com/blogs/machine-learning/announcing-expanded-support-for-extracting-data-from-invoices-and-receipts-using-amazon-textract/
print(
    vision_tool_use_api(
        query="この請求書を書き起こしてください。すべての <document/> フィールドに対して dependentSchemas を適用するよう注意してください。",
        image_url="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2021/07/22/ml3911-img17.jpg",
    )
)

langfuse_context.flush()

# <img src="./images/langfuse-trace-tool-use-vision.png" width="800"/>

## プロンプト管理
### プロンプト管理とは何か？

プロンプト管理は、LLM アプリケーションでプロンプトを保存、バージョン管理、および取得するための体系的なアプローチです。プロンプト管理の重要な側面には、バージョン管理、コードからプロンプトを分離、モニタリング、ロギング、プロンプトの最適化、およびアプリケーションやツールスタックの残りの部分とのプロンプトの統合が含まれます。

Langfuse を使用して、プロンプトを効果的に**管理**および**バージョン管理**します。Langfuse のプロンプト管理は、プロンプト**CMS**（コンテンツ管理システム）です。

### なぜプロンプト管理を使用するのですか？

CMS を使用することの典型的な利点がここに適用されます：

- 分離：アプリケーションを再デプロイせずに新しいプロンプトをデプロイできる。
- 非技術的なユーザーが Langfuse コンソール経由でプロンプトを作成および更新できる。
- プロンプトの以前のバージョンにすぐにロールバックできる。
- 異なるプロンプトバージョンを並べて比較できる。

プラットフォームの利点：

- Langfuse Tracing でプロンプトバージョンのパフォーマンスを追跡できる。

他の実装と比較したパフォーマンス上の利点：

-  プロンプトの最初の使用後、クライアント側のキャッシュと非同期キャッシュ更新により、レイテンシーの影響はない。
-  テキストおよびチャットプロンプトをサポートする。
-  UI、SDK、または API を介して編集/管理できる。

Langfuse でプロンプトを作成するには、いくつかの方法があります：

-  Langfuse コンソール
-  Langfuse SDK
-  Langfuse API

このワークショップでは、Langfuse Python 低レベル SDK を使用して、「モジュール 1 - Amazon Nova モデルを使ったプロンプトエンジニアリング」からのプロンプトの例を再利用してプロンプトを作成します。

In [ ]:
# Langfuse クライアントの初期化
langfuse = Langfuse()

# COT を使わないチャットプロンプトの作成
langfuse.create_prompt(
    name="software-development-project-management-without-COT",
    type="chat",
    prompt=[
        {
            "role": "user",
            "content": "あなたは小さなソフトウェア開発チームのプロジェクトマネージャーです。あなたは開発プロセスを効率化し、タイムリーな納品を実現したいと考えています。",
        }
    ],
    labels=["dev"],
    config={
        "model": MODEL_CONFIG["nova_pro"]["model_id"],
        "maxTokens": MODEL_CONFIG["nova_pro"]["inferenceConfig"]["maxTokens"],
        "temperature": MODEL_CONFIG["nova_pro"]["inferenceConfig"]["temperature"],
    },  # 開発と実験フェーズ用
)

In [ ]:
# COT を使ったチャットプロンプトの作成
langfuse.create_prompt(
    name="software-development-project-management-with-COT",
    type="chat",
    prompt=[
        {
            "role": "user",
            "content": """あなたは小さなソフトウェア開発チームのプロジェクトマネージャーです。あなたは開発プロセスを効率化し、タイムリーな納品を実現したいと考えています。以下の手順に従ってください:\n
{{step1}}\n
\n
{{step2}}\n
\n
{{step3}}\n
\n
{{step4}}\n""",
        }
    ],
    labels=["dev"],
    config={
        "model": MODEL_CONFIG["nova_pro"]["model_id"],
        "maxTokens": MODEL_CONFIG["nova_pro"]["inferenceConfig"]["maxTokens"],
        "temperature": MODEL_CONFIG["nova_pro"]["inferenceConfig"]["temperature"],
    },  # 開発と実験フェーズ用
)

2 つの langfuse プロンプトが正常に作成されているのがわかります。

![langfuse-traces-prompt-management](./images/langfuse-prompt-management.png)


In [ ]:
# 両方のプロンプトを取得し、変数に値を入力してプロンプトを呼び出す
langfuse = Langfuse()

# プロンプトの最新バージョンを取得
sdpm_with_cot_prompt = langfuse.get_prompt(
    "software-development-project-management-with-COT", type="chat", label="dev"
)
# プロンプトテンプレートに変数を挿入
sdpm_with_cot_prompt_compiled = sdpm_with_cot_prompt.compile(
    step1="要件を定義する",
    step2="タスクに分解する",
    step3="期限を設定する",
    step4="進捗を監視し、最適化する",
)

sdpm_without_cot_prompt = langfuse.get_prompt(
    "software-development-project-management-without-COT", type="chat", label="dev"
)
sdpm_without_cot_prompt_compiled = sdpm_without_cot_prompt.compile()

In [ ]:
sdpm_with_cot_prompt_compiled

SDK の呼び出しにプロンプトオブジェクトを追加することで、Langfuse Tracing の生成とプロンプトバージョンをリンクできるようになりました。この連携により、プロンプトのバージョンと名前によるメトリクスの追跡が可能になります。

In [ ]:
# 管理しているプロンプトを用いた会話
@observe()
def main():
    langfuse_context.update_current_trace(
        name="prompt-management-trace",
        user_id="nova-user-1",
        session_id="link-prompt-session",
        tags=["lab1"],
    )

    messages = [
        {
            "role": sdpm_with_cot_prompt_compiled[0]["role"],
            "content": sdpm_with_cot_prompt_compiled[0]["content"],
        }
    ]

    simple_chat(
        model_config=MODEL_CONFIG["nova_pro"],
        messages=messages,
        prompt=sdpm_with_cot_prompt,
    )

    messages = [
        {
            "role": sdpm_without_cot_prompt_compiled[0]["role"],
            "content": sdpm_without_cot_prompt_compiled[0]["content"],
        }
    ]

    simple_chat(
        model_config=MODEL_CONFIG["nova_pro"],
        messages=messages,
        prompt=sdpm_without_cot_prompt,
    )


main()
langfuse_context.flush()

トレースがプロンプトのバージョンとプロンプト名にリンクしていることがわかります。

![langfuse-traces-prompt-management](./images/langfuse-link-prompt.png)

## Lab1 まとめ：
Lab1 では、Langfuse と AWS を統合してプロンプトトレースを効果的に管理するための基本を探求しました。
トレースの更新方法、プロンプトをユーザーセッションにリンクする方法、そしてこれらのリンクを実際の例で視覚化する方法を示しました。

このラボを締めくくるにあたり、獲得した基礎スキルについて少し考えてみてください。
今、AWS のイベントに参加している場合は、次のラボに進む前に、ワークショップスタジオに戻って追加の指示を確認してください。次のラボでは、RAG 関連のトレーシングと評価についてさらに深く掘り下げます。